In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# === LOAD & PREP DATA ===
games = pd.read_csv("games_full.csv")
games = games.dropna(subset=['title'])
all_titles = sorted(games['title'].drop_duplicates().tolist())

# === WIDGETS ===
search_box = widgets.Combobox(
    placeholder='Type a game title here...',
    options=all_titles,
    description='Search:',
    ensure_option=True,
    layout=widgets.Layout(width='70%')
)

add_button = widgets.Button(description='Add Game')
reset_button = widgets.Button(description='Reset', button_style='danger')
confirm_button = widgets.Button(description='Get Recommendations', button_style='success')

liked_list = widgets.SelectMultiple(
    options=[],
    description='Liked Games:',
    rows=8,
    layout=widgets.Layout(width='70%')
)

output = widgets.Output()

# === CALLBACK: ADD GAME ===
def on_add_click(b):
    game = search_box.value.strip()
    if game in all_titles and game not in liked_list.options:
        liked_list.options = list(liked_list.options) + [game]
        search_box.value = ""

# === CALLBACK: RESET ===
def on_reset_click(b):
    liked_list.options = []
    search_box.value = ""
    with output:
        output.clear_output()

# === CALLBACK: RECOMMEND ===
def on_confirm_click(b):
    with output:
        output.clear_output()
        liked_titles = list(liked_list.options)
        
        if not liked_titles:
            print("ERROR: Please add at least one game before getting recommendations.")
            return
        
        print("✅ You selected ✅")
        for title in liked_titles:
            print(f"• {title}")
        
        # Label dataset
        games['liked'] = games['title'].apply(lambda x: 1 if x in liked_titles else 0)

        # Combine text features
        text_cols = ['genres', 'supported_languages', 'developers', 'publishers']
        games[text_cols] = games[text_cols].fillna("")
        games['text_features'] = games[text_cols].agg(' '.join, axis=1)

        # Balance dataset (more realistic)
        liked = games[games['liked'] == 1]
        not_liked = games[games['liked'] == 0].sample(n=min(len(games[games['liked'] == 0]), len(liked) * 3), random_state=42)
        games_balanced = pd.concat([liked, not_liked]).copy()

        # TF-IDF Vectorizer
        vectorizer = TfidfVectorizer()
        X_train = vectorizer.fit_transform(games_balanced['text_features'])
        y_train = games_balanced['liked']

        # Train model
        model = MultinomialNB()
        model.fit(X_train, y_train)

        # Predict on all games
        X_all = vectorizer.transform(games['text_features'])
        games['predicted_like'] = model.predict_proba(X_all)[:, 1]

        # Get recommendations (excluding already liked)
        recommendations = games[~games['title'].isin(liked_titles)].sort_values('predicted_like', ascending=False)

        print("\n🎮 Top 10 Game Recommendations 🎮")
        for i, row in recommendations[['title', 'predicted_like']].head(10).iterrows():
            print(f"• {row['title']} (Score: {row['predicted_like']:.2f})")

# === PREVENT DUPLICATE CALLBACKS ===
try:
    add_button._click_callbacks.callbacks.clear()
    reset_button._click_callbacks.callbacks.clear()
    confirm_button._click_callbacks.callbacks.clear()
except Exception:
    pass

add_button.on_click(on_add_click)
reset_button.on_click(on_reset_click)
confirm_button.on_click(on_confirm_click)

# === DISPLAY UI ===
ui = widgets.VBox([
    widgets.HBox([search_box, add_button, reset_button]),
    liked_list,
    confirm_button,
    output
])

display(ui)
